In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/WUSTL/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9998827239867143,
        0.9600649800293866, array([1, 1, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9996984331086939,
        0.9305545272093338, array([3, 3, 3, ..., 3, 3, 3])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.9984796002563319,
        0.7518383136072314, array([5, 5, 5, ..., 5, 5, 5])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9998701586995765,
        0.939002641264322, array([1, 1, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9971518682487759,
        0.8894642081911899, array([3, 3, 3, ..., 3, 3, 3])],
       ['FGSM', 0.3, 'Def

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    # ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    # ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1 and attack_name!= "CaFA" and attack_name!= "AutoPGD":
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '7' encode 6
defense '8' encode 7
defense '9' encode 8
defense '10' encode 9


In [5]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import StandardScaler
from scipy.stats import entropy
from tqdm import tqdm, tqdm_notebook
from joblib import Parallel, delayed
import warnings

def stratified_random_sample(X, y, fraction=None, count=None, random_state=42):
    assert (fraction is not None) ^ (count is not None), "Specify either fraction or count"

    rng = np.random.RandomState(seed=random_state)
    X_out, y_out, indices_out = [], [], []

    for label in np.unique(y):
        X_c = X[y == label]
        y_c = y[y == label]
        indices_c = np.where(y == label)[0]

        n_total = len(X_c)
        n_sample = int(n_total * fraction) if fraction is not None else min(count, n_total)
        # Ensure at least one sample per class
        n_sample = max(1, n_sample)

        idx = rng.choice(n_total, size=n_sample, replace=False)

        X_out.append(X_c[idx])
        y_out.append(y_c[idx])
        indices_out.append(indices_c[idx])

    return np.concatenate(X_out, axis=0), np.concatenate(y_out, axis=0), np.concatenate(indices_out, axis=0)

def compute_entropy(prob):
    return entropy(prob, base=2)

def eoal_sampling(X_pool, y_pool, model, fraction, known_class, diversity=True, random_state=42, use_notebook=False):

    np.random.seed(random_state)
    
    # Standardize features for better clustering
    scaler = StandardScaler()
    X_pool_scaled = scaler.fit_transform(X_pool)
    
    # Get model predictions and probabilities
    softprobs = model.predict_proba(X_pool)
    y_pred = model.predict(X_pool)
    
    # Parallel compute entropy for large datasets
    entropy_list = np.array(Parallel(n_jobs=-1)(delayed(compute_entropy)(prob) for prob in softprobs))
    
    # Identify known and unknown samples
    known_mask = y_pred < known_class
    queryIndex = np.arange(len(X_pool))
    queryIndex_k = queryIndex[known_mask]
    uncertaintyArr_k = entropy_list[known_mask]
    labelArr_k = y_pool[known_mask]
    
    # Calculate number of samples to select
    query_batch = min(int(fraction * len(X_pool)), len(queryIndex_k))  # Ensure query_batch <= len(queryIndex_k)
    
    if not diversity:
        # Uncertainty-based sampling only
        sorted_idx = np.argsort(uncertaintyArr_k)[-query_batch:]  # Select top uncertain samples
        selected_idx = queryIndex_k[sorted_idx]
        selected_gt = labelArr_k[sorted_idx]
    else:
        # Diversity-based sampling with MiniBatchKMeans clustering
        embeddings_k = X_pool_scaled[known_mask]
        # Estimate number of clusters
        num_clusters = min(known_class, query_batch, len(embeddings_k))
        if num_clusters < 1:
            num_clusters = 1
        kmeans = MiniBatchKMeans(n_clusters=num_clusters, random_state=random_state, batch_size=1000)
        cluster_labels = kmeans.fit_predict(embeddings_k)
        
        # Allocate budget per cluster
        rem = min(query_batch, len(queryIndex_k))
        num_per_cluster = max(1, rem // num_clusters)  # Ensure at least 1 sample per cluster
        selected_idx = []
        selected_gt = []
        ax = [0] * num_clusters
        
        # Initialize tqdm progress bar
        tqdm_func = tqdm_notebook if use_notebook else tqdm
        with tqdm_func(total=rem, desc="Sampling Progress", unit="samples") as pbar:
            while rem > 0:
                for cls in range(num_clusters):
                    temp_ent = uncertaintyArr_k[cluster_labels == cls]
                    temp_index = queryIndex_k[cluster_labels == cls]
                    temp_gt = labelArr_k[cluster_labels == cls]
                    # Limit samples to remaining budget
                    samples_to_take = min(num_per_cluster, len(temp_ent), rem)
                    if samples_to_take > 0:
                        sorted_idx = np.argsort(temp_ent)[-samples_to_take:]
                        ax[cls] += len(sorted_idx)
                        rem -= len(sorted_idx)
                        selected_idx.extend(temp_index[sorted_idx])
                        selected_gt.extend(temp_gt[sorted_idx])
                        pbar.update(len(sorted_idx))  # Update progress bar
                pbar.set_description(f"Sampling Progress (Remaining: {rem})")
                if rem <= 0:
                    break  # Exit loop if budget exhausted
        
        selected_idx = np.array(selected_idx)
        selected_gt = np.array(selected_gt)
    
    # Handle remaining budget with unknown samples
    if len(selected_gt) < query_batch:
        rem_budget = query_batch - len(selected_idx)
        uncertaintyArr_u = entropy_list[~known_mask]
        queryIndex_u = queryIndex[~known_mask]
        labelArr_u = y_pool[~known_mask]
        
        sorted_idx_extra = np.argsort(uncertaintyArr_u)[-rem_budget:]
        selected_idx = np.concatenate((selected_idx, queryIndex_u[sorted_idx_extra]))
        selected_gt = np.concatenate((selected_gt, labelArr_u[sorted_idx_extra]))
    
    # Calculate precision and recall
    Len_labeled_ind_train = len(np.where(y_pool < known_class)[0])  # Assume all known samples in pool
    precision = len(np.where(selected_gt < known_class)[0]) / len(selected_gt) if len(selected_gt) > 0 else 0
    recall = len(np.where(selected_gt < known_class)[0]) / (
        len(np.where(y_pool < known_class)[0]) + Len_labeled_ind_train) if (
        len(np.where(y_pool < known_class)[0]) + Len_labeled_ind_train) > 0 else 0
    
    return selected_idx, selected_gt, precision, recall

def batch_mode_active_learning(X_pool, y_pool, model, fraction=0.5, random_state=42, use_notebook=False):

    known_class = len(np.unique(y_pool))  # Assume all classes in y_pool are known for simplicity
    selected_idx, selected_gt, _, _ = eoal_sampling(
        X_pool, y_pool, model, fraction, known_class, diversity=True, random_state=random_state, use_notebook=use_notebook
    )
    
    X_selected = X_pool[selected_idx]
    y_selected = selected_gt
    remaining_mask = np.ones(len(X_pool), dtype=bool)
    remaining_mask[selected_idx] = False
    X_remaining = X_pool[remaining_mask]
    y_remaining = y_pool[remaining_mask]
    
    return X_remaining, X_selected, y_remaining,y_selected

X_pool = x_test_adv_0_1 
y_pool = y_label_0_1
X_train_init, y_train_init, _ = stratified_random_sample(X_pool, y_pool, fraction=0.1, random_state=42)

clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_init, y_train_init)
# Perform batch mode active learning with fractions 50%, 10%, 1%
_, X_10, _, y_10 = batch_mode_active_learning(
    x_test_adv_0_1, y_label_0_1, clf, fraction=0.1, random_state=42)

Sampling Progress (Remaining: 0): 100%|██████████| 167127/167127 [00:00<00:00, 436248.71samples/s]   


In [6]:
X_10.shape

(167127, 42)

In [7]:
np.unique(y_10, return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([100852,     12,  21682,     20,   1313,  40025,      3,   1005,
            51,   2164]))

In [8]:
from xgboost import XGBClassifier

print('xgb')
xgb10 = XGBClassifier()
xgb10.fit(X_10, y_10)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf10 = RandomForestClassifier()
rf10.fit(X_10, y_10)

# print("DT")
# from sklearn.tree import DecisionTreeClassifier
# dt10 = DecisionTreeClassifier()
# dt10.fit(X_10, y_10)

xgb
RF


RandomForestClassifier()

In [9]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

In [10]:


base_path_template = "/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = xgb10.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ActiveLearning_EOAL_Exclude2Attack/WUSTL_Input10/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ActiveLearning_EOAL_Exclude2Attack/WUSTL_Input10/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[2 3 5 6] [    13      1      1 238738]
(13, 42) (13,)
Save 2 to baseline_Def2.npy
(1, 42) (1,)
Save 3 to baseline_Def3.npy
(1, 42) (1,)
Save 5 to baseline_Def5.npy
(238738, 42) (238738,)
Save 6 to baseline_Def6.npy
Execution Time: 0.398340 seconds
(716259,)
[ 1  3  5  6  7  9 10] [448394  19648    911 233606      1  13521    178]
(448394, 42) (448394,)
Save 1 to BIM_Def1.npy
(19648, 42) (19648,)
Save 3 to BIM_Def3.npy
(911, 42) (911,)
Save 5 to BIM_Def5.npy
(233606, 42) (233606,)
Save 6 to BIM_Def6.npy
(1, 42) (1,)
Save 7 to BIM_Def7.npy
(13521, 42) (13521,)
Save 9 to BIM_Def9.npy
(178, 42) (178,)
Save 10 to BIM_Def10.npy
Execution Time: 0.947357 seconds
(716259,)
[ 1  2  3  5  6  7  8  9 10] [444470      5   6452   8827 249423      1    593   1702   4786]
(444470, 42) (444470,)
Save 1 to FGSM_Def1.npy
(5, 42) (5,)
Save 2 to FGSM_Def2.npy
(6452, 42) (6452,)
Save 3 to FGSM_Def3.npy
(8827, 42) (8827,)
Save 5 to FGSM_Def5.npy
(249423, 42) (249423,)
Save 6 to FGSM_Def6.npy
(1, 4

In [11]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = rf10.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning_EOAL_Exclude2Attack/WUSTL_Input10/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning_EOAL_Exclude2Attack/WUSTL_Input10/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[2 3 5 6] [    13      1      1 238738]
(13, 42) (13,)
Save 2 to baseline_Def2.npy
(1, 42) (1,)
Save 3 to baseline_Def3.npy
(1, 42) (1,)
Save 5 to baseline_Def5.npy
(238738, 42) (238738,)
Save 6 to baseline_Def6.npy
Execution Time: 0.884625 seconds
(716259,)
[ 1  2  3  5  6 10] [  1212   1092  32704   6108 675071     72]
(1212, 42) (1212,)
Save 1 to BIM_Def1.npy
(1092, 42) (1092,)
Save 2 to BIM_Def2.npy
(32704, 42) (32704,)
Save 3 to BIM_Def3.npy
(6108, 42) (6108,)
Save 5 to BIM_Def5.npy
(675071, 42) (675071,)
Save 6 to BIM_Def6.npy
(72, 42) (72,)
Save 10 to BIM_Def10.npy
Execution Time: 3.013117 seconds
(716259,)
[ 1  2  3  5  6  9 10] [    30    606   7980  13541 692159      5   1938]
(30, 42) (30,)
Save 1 to FGSM_Def1.npy
(606, 42) (606,)
Save 2 to FGSM_Def2.npy
(7980, 42) (7980,)
Save 3 to FGSM_Def3.npy
(13541, 42) (13541,)
Save 5 to FGSM_Def5.npy
(692159, 42) (692159,)
Save 6 to FGSM_Def6.npy
(5, 42) (5,)
Save 9 to FGSM_Def9.npy
(1938, 42) (1938,)
Save 10 to FGSM_Def10.n

In [12]:
# from sklearn.model_selection import train_test_split

# for attack_name, epsilons in attack_models:
#     start_time = time.time()
#     all_adv_examples = []
#     all_y = []
    
#     if attack_name != "baseline":
#         for epsilon in epsilons:
#             filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
#             x_test_adv = np.load(filename)
#             all_adv_examples.append(x_test_adv)
#             all_y.append(y_test)
    
#         X = np.concatenate(all_adv_examples, axis=0)
#         y = np.concatenate(all_y, axis=0)
#     else:
#         X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
#         y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

#     y_pred = dt10.predict(X)
#     print(y_pred.shape)
#     y_pred = label_encoder.inverse_transform(y_pred)
#     unique_classes, count = np.unique(y_pred, return_counts=True)
#     print(unique_classes, count)
#     for class_num in unique_classes:
#         indices = np.where(y_pred == class_num)[0]
#         X_class = X[indices]
#         y_class = y[indices]
#         print(X_class.shape, y_class.shape)
    
#         x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT_ActiveLearning_EOAL_Exclude2Attack/WUSTL_Input10/x_test_adv_{attack_name}_Def{class_num}.npy"
#         y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT_ActiveLearning_EOAL_Exclude2Attack/WUSTL_Input10/y_test_adv_{attack_name}_Def{class_num}.npy"
        
#         np.save(x_filename, X_class)
#         np.save(y_filename, y_class)
    
#         print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

#     end_time = time.time()
#     result = end_time - start_time
#     print(f"Execution Time: {result:.6f} seconds")
    

In [13]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}10")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/{m_name}_ActiveLearning_EOAL_Exclude2Attack/WUSTL_Input10/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [14]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input10_ActiveLearning_EOAL_Exclude2Attack.csv")

In [15]:
all_attack_list

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,XGB10,0,13,1,0,1,238738,0,0,0,0,0,238753
1,XGB10,448394,0,19648,0,911,233606,1,0,13521,178,0,716259
2,XGB10,444470,5,6452,0,8827,249423,1,593,1702,4786,0,716259
3,XGB10,448394,0,19648,0,911,233606,1,0,13521,178,0,716259
4,XGB10,94,0,6113,16,9286,688127,7,3208,203,9205,0,716259
5,XGB10,271627,15,137790,4,1369,292303,4,6,12960,181,0,716259
6,XGB10,0,35,0,0,3,716221,0,0,0,0,0,716259
7,XGB10,216015,11051,14743,0,9989,440517,1853,1,17954,4136,0,716259
8,XGB10,5501,0,464725,0,161,237517,0,0,8355,0,0,716259
9,XGB10,424710,0,32140,0,3424,248768,3,0,7211,3,0,716259
